In [1]:
import requests
import json

In [2]:
API_KEY = ''

In [1]:
# Function to fetch recent matches from Opendota API, it only retrieves 100 matches at a time so there is a script to
# Change the seed of matches to the current match id, it starts by taking the freshest then works its way down
def fetch_recent_matches():
    matches = []
    num_matches_fetched = 0
    seen_match_ids = set()
    current_match_id = None

    while num_matches_fetched < 5000:
        url = f'https://api.opendota.com/api/publicMatches?api_key={API_KEY}&min_rank=80'
        if current_match_id:
            url += f'&less_than_match_id={current_match_id}'

        response = requests.get(url)
        
        if response.status_code == 200:
            new_matches = response.json()
            if not new_matches:
                break
            
            for match in new_matches:
                match_id = match['match_id']
                if match_id not in seen_match_ids:
                    matches.append(match)
                    seen_match_ids.add(match_id)
                    num_matches_fetched += 1
                    current_match_id = match_id
        else:
            print(f"Error fetching recent matches: {response.status_code} - {response.text}")
            break
    
    return matches

In [3]:
# A function to fetch invdividual match details and do some data cleaning in it since each match id contains a lot of data, most of
# it being useless for this project
def fetch_match_details(match_id):
    url = f'https://api.opendota.com/api/matches/{match_id}?api_key={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        match_details = response.json()
        
        # Define columns to drop
        columns_to_drop = ['match_id', 'chat', 'cosmetics', 'objectives', 'players', 'pre_game_duration', 'start_time',
                          'match_seq_num', 'series_id', 'series_type', 'replay_salt', 'all_word_counts', 'my_word_counts',
                          'throw', 'loss', 'replay_url', 'comeback', 'stomp', 'metadata', 'version', 'draft_timings', 'teamfights',
                          'radiant_gold_adv', 'radiant_xp_adv', 'engine', 'cluster', 'lobby_type', 'human_players', 'leagueid', 
                           'game_mode', 'flags', 'patch', 'first_blood_time', 'od_data']
        
        # Drop specified columns
        for col in columns_to_drop:
            match_details.pop(col, None)
        
        return match_details
    else:
        print(f"Error fetching match details for match {match_id}: {response.status_code} - {response.text}")
        return None

In [4]:
# A function to filter the matches from the fetched matches to only include matches with Ranked All pick as their game mode
def filter_matches(matches):
    filtered_matches = []
    for match in matches:
        if match['lobby_type'] == 7 and match['game_mode'] == 22:  # Ranked and All Pick
            filtered_matches.append(match)
    return filtered_matches

In [5]:
# A functino to write and save the data to file
def save_to_file(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file)

In [7]:
output_filename = 'recent_matches.json'  # Output file name
with open(output_filename, 'r') as file:
    recent_matches = json.load(file)

In [9]:
# Utilizing the above functions to retrieve the data
recent_matches = fetch_recent_matches()

if recent_matches:
    # Save recent matches data to a file
    recent_matches_filename = 'recent_matches.json'
    save_to_file(recent_matches, recent_matches_filename)
    print(f"{len(recent_matches)} recent matches saved to '{recent_matches_filename}'")
else:
    print("No recent matches fetched.")

if recent_matches:
    # Filter matches
    filtered_matches = filter_matches(recent_matches)

    # Fetch details of filtered matches
    filtered_matches_details = []
    for match in filtered_matches:
        match_details = fetch_match_details(match['match_id'])
        if match_details:
            filtered_matches_details.append(match_details)

    # Save filtered matches data to a file
    filename = 'filtered_matches.json'
    with open(filename, 'w') as file:
        json.dump(filtered_matches_details, file)

    print(f"{len(filtered_matches)} matches saved to '{filename}'")
else:
    print("No recent matches fetched.")

5000 recent matches saved to 'recent_matches.json'
4875 matches saved to 'filtered_matches.json'
